## Imports

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import datetime
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import io
import os

## Load Data

In [24]:
train = pd.read_csv('./data/train.csv')
print(f"Training data size is {train.shape}")

Training data size is (42000, 785)


## Data Preprocessing

In [25]:
X = train.drop('label', axis=1).values
y = train['label'].values
X = X / 255.0
X = X.reshape(-1,28,28,1)
y = to_categorical(y)

print(f"Label size {y.shape}")

Label size (42000, 10)


In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((37800, 28, 28, 1), (4200, 28, 28, 1), (37800, 10), (4200, 10))

## Data Augmentation

In [27]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.01, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

train_gen = datagen.flow(X_train, y_train, batch_size=128)
test_gen = datagen.flow(X_val, y_val, batch_size=128)

## Model

### Costum 

In [28]:
model=Sequential()
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 conv2d_8 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 12, 12, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 128)        

### Lenet

In [29]:
# from keras.layers import Conv2D, AveragePooling2D
# 
# model=Sequential()
# model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(28,28,1)))
# model.add(AveragePooling2D())
# model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'))
# model.add(AveragePooling2D())
# model.add(Flatten())
# model.add(Dense(units=120, activation='tanh'))
# model.add(Dense(units=84, activation='tanh'))
# model.add(Dense(units=10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 
# model.summary()

In [30]:
epochs = 100
batch_size = 128
train_steps = X_train.shape[0] // batch_size
valid_steps = X_val.shape[0] // batch_size
es = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        verbose=1,
        mode="max",
        restore_best_weights=True,
     )
rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.00001,
     )

## Tensorboard

In [31]:
classes=[0,1,2,3,4,5,6,7,8,9]

def log_confusion_matrix(epoch, logs):
  y_pred = model.predict(X_val) # Predict class probabilities as 2 => [0.1, 0, 0.9, 0, 0, 0, 0, 0, 0, 0]
  Y_pred = np.argmax(y_pred, 1) # Decode Predicted labels
  Y_test = np.argmax(y_val, 1) # Decode labels
  con_mat = confusion_matrix(Y_test, Y_pred) # Confusion matrix

  con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

  con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes,
                     columns = classes)

  figure = plt.figure(figsize=(8, 8))
  sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

  buf = io.BytesIO()
  plt.savefig(buf, format='png')

  plt.close(figure)
  buf.seek(0)
  image = tf.image.decode_png(buf.getvalue(), channels=4)

  image = tf.expand_dims(image, 0)

  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", image, step=epoch)

In [32]:
logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir = logdir, histogram_freq = 1)
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

# Command for starting Tensorboard
# python -m tensorboard.main --logdir=logs

## Run

In [33]:
# Data Augmentation inclusive
history = model.fit(train_gen,
                    epochs = epochs,
                    steps_per_epoch = train_steps,
                    validation_data = test_gen, 
                    validation_steps = valid_steps,
                    callbacks=[es, rp, tensorboard_callback,cm_callback])


# Data Augmentation exclusive
# history = model.fit(
#     x=X_train,  # Direkte Verwendung der Trainingsdaten
#     y=y_train,  # Direkte Verwendung der Trainingslabels
#     epochs=epochs,
#     batch_size=batch_size,  
#     validation_data=(X_val, y_val),
#     callbacks=[es, rp, tensorboard_callback, cm_callback]
# )


Epoch 1/100
295/295 [==============================] - 69s 231ms/step - loss: 0.1620 - accuracy: 0.9491 - val_loss: 1.1363 - val_accuracy: 0.6270 - lr: 0.0010
Epoch 2/100
295/295 [==============================] - 69s 235ms/step - loss: 0.0643 - accuracy: 0.9806 - val_loss: 0.0800 - val_accuracy: 0.9768 - lr: 0.0010
Epoch 3/100
295/295 [==============================] - 70s 236ms/step - loss: 0.0482 - accuracy: 0.9848 - val_loss: 0.0716 - val_accuracy: 0.9761 - lr: 0.0010
Epoch 4/100
295/295 [==============================] - 70s 238ms/step - loss: 0.0435 - accuracy: 0.9859 - val_loss: 0.0559 - val_accuracy: 0.9824 - lr: 0.0010
Epoch 5/100
295/295 [==============================] - 72s 245ms/step - loss: 0.0402 - accuracy: 0.9877 - val_loss: 0.1476 - val_accuracy: 0.9524 - lr: 0.0010
Epoch 6/100
295/295 [==============================] - 70s 238ms/step - loss: 0.0317 - accuracy: 0.9899 - val_loss: 0.0482 - val_accuracy: 0.9866 - lr: 0.0010
Epoch 7/100
295/295 [=========================

In [34]:
# 1 - Save the full model
model.save("./data/trained_models/name.h5")
model.save("./data/trained_models/name_TFSaveFormat")
# 2 - Save the weights of the model
#model.save_weights("./data/trained_models/name_weights.h5")
# 3 - Save the architecture of the model
# json_string = model.to_json()
# with open("./data/trained_models/name_model.h5", "w") as f:
#     f.write(json_string)

INFO:tensorflow:Assets written to: ./data/trained_models/name_TFSaveFormat\assets


INFO:tensorflow:Assets written to: ./data/trained_models/name_TFSaveFormat\assets
